## 1. Import Libraries and Setup

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Sequential, Model

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print("Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load dataset
df = pd.read_csv('../../../data/data/Churn_Modeling.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data information
print("Dataset Information:")
print(df.info())
print("\n" + "="*80)
print("\nStatistical Summary:")
df.describe()

In [ ]:
# Check target distribution
print("Target Variable Distribution:")
print(df['Exited'].value_counts())
print(f"\nChurn Rate: {df['Exited'].mean()*100:.2f}%")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
df['Exited'].value_counts().plot(kind='bar', ax=ax1, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Exited (0=Stayed, 1=Churned)', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')
ax1.set_xticklabels(['Stayed', 'Churned'], rotation=0)
ax1.grid(axis='y', alpha=0.3)

# Pie chart
labels = ['Stayed', 'Churned']
sizes = df['Exited'].value_counts()
colors = ['#66b3ff', '#ff9999']
ax2.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90,
       textprops={'fontsize': 12, 'fontweight': 'bold'})
ax2.set_title('Churn Percentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values found!")

## 3. Exploratory Data Analysis

In [ ]:
# Feature distributions by churn status
numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, feature in enumerate(numerical_features):
    df[df['Exited'] == 0][feature].hist(bins=30, alpha=0.6, label='Stayed', 
                                         ax=axes[idx], edgecolor='black')
    df[df['Exited'] == 1][feature].hist(bins=30, alpha=0.6, label='Churned', 
                                         ax=axes[idx], edgecolor='black')
    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].set_title(f'{feature} Distribution by Churn', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Categorical features analysis
categorical_features = ['Geography', 'Gender']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, feature in enumerate(categorical_features):
    churn_by_cat = df.groupby([feature, 'Exited']).size().unstack()
    churn_by_cat.plot(kind='bar', ax=axes[idx], edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(feature, fontsize=12)
    axes[idx].set_ylabel('Count', fontsize=12)
    axes[idx].set_title(f'Churn Distribution by {feature}', fontsize=14, fontweight='bold')
    axes[idx].legend(['Stayed', 'Churned'])
    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha='right')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(14, 10))

# Select numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
correlation = df[numerical_cols].corr()

sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Correlation with target
print("\nCorrelation with Churn (Exited):")
print(correlation['Exited'].sort_values(ascending=False))

## 4. Data Preprocessing

In [ ]:
# Drop irrelevant columns
df_clean = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Encode categorical variables
# Geography: One-hot encoding
geography_encoded = pd.get_dummies(df_clean['Geography'], prefix='Geography', drop_first=False)

# Gender: Label encoding
gender_encoder = LabelEncoder()
df_clean['Gender'] = gender_encoder.fit_transform(df_clean['Gender'])

# Combine
df_encoded = pd.concat([df_clean.drop('Geography', axis=1), geography_encoded], axis=1)

print(f"Encoded dataset shape: {df_encoded.shape}")
print(f"\nFeatures: {df_encoded.columns.tolist()}")

In [ ]:
# Separate features and target
X = df_encoded.drop('Exited', axis=1)
y = df_encoded['Exited']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature names: {X.columns.tolist()}")

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining churn rate: {y_train.mean()*100:.2f}%")
print(f"Test churn rate: {y_test.mean()*100:.2f}%")

In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaled training data shape: {X_train_scaled.shape}")
print(f"Scaled test data shape: {X_test_scaled.shape}")
print(f"\nScaled data statistics (train):")
print(f"Mean: {X_train_scaled.mean():.4f}")
print(f"Std: {X_train_scaled.std():.4f}")

## 5. Build Baseline Neural Network

In [ ]:
# Define baseline model
def create_baseline_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim, name='dense_1'),
        Dense(32, activation='relu', name='dense_2'),
        Dense(16, activation='relu', name='dense_3'),
        Dense(1, activation='sigmoid', name='output')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Create model
baseline_model = create_baseline_model(X_train_scaled.shape[1])

print("Baseline Model Architecture:")
baseline_model.summary()

In [ ]:
# Train baseline model
print("Training Baseline Model...")

history_baseline = baseline_model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\nBaseline model training complete!")

In [ ]:
# Evaluate baseline model
baseline_results = baseline_model.evaluate(X_test_scaled, y_test, verbose=0)
print("Baseline Model Test Results:")
print(f"Loss: {baseline_results[0]:.4f}")
print(f"Accuracy: {baseline_results[1]:.4f}")
print(f"AUC: {baseline_results[2]:.4f}")

## 6. Improved Model with Dropout and Batch Normalization

In [ ]:
# Define improved model
def create_improved_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim, name='dense_1'),
        BatchNormalization(name='bn_1'),
        Dropout(0.3, name='dropout_1'),
        
        Dense(64, activation='relu', name='dense_2'),
        BatchNormalization(name='bn_2'),
        Dropout(0.3, name='dropout_2'),
        
        Dense(32, activation='relu', name='dense_3'),
        BatchNormalization(name='bn_3'),
        Dropout(0.2, name='dropout_3'),
        
        Dense(16, activation='relu', name='dense_4'),
        Dropout(0.2, name='dropout_4'),
        
        Dense(1, activation='sigmoid', name='output')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# Create improved model
improved_model = create_improved_model(X_train_scaled.shape[1])

print("Improved Model Architecture:")
improved_model.summary()

In [ ]:
# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = callbacks.ModelCheckpoint(
    '../../../data/outputs/best_churn_model.keras',
    monitor='val_auc',
    mode='max',
    save_best_only=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

print("Callbacks configured:")
print("- EarlyStopping (patience=10)")
print("- ModelCheckpoint (best AUC)")
print("- ReduceLROnPlateau (factor=0.5, patience=5)")

In [ ]:
# Train improved model
print("\nTraining Improved Model...")

history_improved = improved_model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

print("\nImproved model training complete!")

In [ ]:
# Evaluate improved model
improved_results = improved_model.evaluate(X_test_scaled, y_test, verbose=0)
print("Improved Model Test Results:")
print(f"Loss: {improved_results[0]:.4f}")
print(f"Accuracy: {improved_results[1]:.4f}")
print(f"AUC: {improved_results[2]:.4f}")

## 7. Training History Visualization

In [ ]:
# Plot training history
def plot_history(history, title_prefix):
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    
    # Loss
    axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title(f'{title_prefix} - Loss Curves', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Accuracy
    axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title(f'{title_prefix} - Accuracy Curves', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    # AUC
    axes[2].plot(history.history['auc'], label='Training AUC', linewidth=2)
    axes[2].plot(history.history['val_auc'], label='Validation AUC', linewidth=2)
    axes[2].set_xlabel('Epoch', fontsize=12)
    axes[2].set_ylabel('AUC', fontsize=12)
    axes[2].set_title(f'{title_prefix} - AUC Curves', fontsize=14, fontweight='bold')
    axes[2].legend()
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot baseline history
print("Baseline Model Training History:")
plot_history(history_baseline, 'Baseline Model')

# Plot improved history
print("\nImproved Model Training History:")
plot_history(history_improved, 'Improved Model')

## 8. Model Evaluation and Comparison

In [ ]:
# Generate predictions
y_pred_baseline = (baseline_model.predict(X_test_scaled) > 0.5).astype(int)
y_pred_improved = (improved_model.predict(X_test_scaled) > 0.5).astype(int)

y_pred_proba_baseline = baseline_model.predict(X_test_scaled)
y_pred_proba_improved = improved_model.predict(X_test_scaled)

print("Predictions generated for both models.")

In [ ]:
# Classification reports
print("BASELINE MODEL - Classification Report:")
print("="*80)
print(classification_report(y_test, y_pred_baseline, target_names=['Stayed', 'Churned']))

print("\n" + "="*80)
print("IMPROVED MODEL - Classification Report:")
print("="*80)
print(classification_report(y_test, y_pred_improved, target_names=['Stayed', 'Churned']))

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Baseline
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', ax=axes[0],
           xticklabels=['Stayed', 'Churned'], yticklabels=['Stayed', 'Churned'])
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_title('Baseline Model - Confusion Matrix', fontsize=14, fontweight='bold')

# Improved
cm_improved = confusion_matrix(y_test, y_pred_improved)
sns.heatmap(cm_improved, annot=True, fmt='d', cmap='Greens', ax=axes[1],
           xticklabels=['Stayed', 'Churned'], yticklabels=['Stayed', 'Churned'])
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('Actual', fontsize=12)
axes[1].set_title('Improved Model - Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves
fpr_baseline, tpr_baseline, _ = roc_curve(y_test, y_pred_proba_baseline)
fpr_improved, tpr_improved, _ = roc_curve(y_test, y_pred_proba_improved)

roc_auc_baseline = roc_auc_score(y_test, y_pred_proba_baseline)
roc_auc_improved = roc_auc_score(y_test, y_pred_proba_improved)

plt.figure(figsize=(10, 8))
plt.plot(fpr_baseline, tpr_baseline, linewidth=2, label=f'Baseline (AUC = {roc_auc_baseline:.4f})')
plt.plot(fpr_improved, tpr_improved, linewidth=2, label=f'Improved (AUC = {roc_auc_improved:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Baseline Model ROC-AUC: {roc_auc_baseline:.4f}")
print(f"Improved Model ROC-AUC: {roc_auc_improved:.4f}")

In [ ]:
# Precision-Recall Curves
precision_baseline, recall_baseline, _ = precision_recall_curve(y_test, y_pred_proba_baseline)
precision_improved, recall_improved, _ = precision_recall_curve(y_test, y_pred_proba_improved)

ap_baseline = average_precision_score(y_test, y_pred_proba_baseline)
ap_improved = average_precision_score(y_test, y_pred_proba_improved)

plt.figure(figsize=(10, 8))
plt.plot(recall_baseline, precision_baseline, linewidth=2, 
        label=f'Baseline (AP = {ap_baseline:.4f})')
plt.plot(recall_improved, precision_improved, linewidth=2, 
        label=f'Improved (AP = {ap_improved:.4f})')
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curves Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Baseline Model Average Precision: {ap_baseline:.4f}")
print(f"Improved Model Average Precision: {ap_improved:.4f}")

## 9. Compare Different Optimizers

In [ ]:
# Test different optimizers
optimizers_to_test = {
    'Adam': optimizers.Adam(learning_rate=0.001),
    'RMSprop': optimizers.RMSprop(learning_rate=0.001),
    'SGD': optimizers.SGD(learning_rate=0.01, momentum=0.9)
}

optimizer_results = {}

for opt_name, optimizer in optimizers_to_test.items():
    print(f"\nTraining with {opt_name} optimizer...")
    
    # Create model
    model = create_improved_model(X_train_scaled.shape[1])
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    # Train
    history = model.fit(
        X_train_scaled, y_train,
        epochs=30,
        batch_size=32,
        validation_split=0.2,
        verbose=0
    )
    
    # Evaluate
    results = model.evaluate(X_test_scaled, y_test, verbose=0)
    optimizer_results[opt_name] = {
        'loss': results[0],
        'accuracy': results[1],
        'auc': results[2],
        'history': history
    }
    
    print(f"{opt_name} - Loss: {results[0]:.4f}, Accuracy: {results[1]:.4f}, AUC: {results[2]:.4f}")

print("\nOptimizer comparison complete!")

In [ ]:
# Visualize optimizer comparison
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

metrics = ['loss', 'accuracy', 'auc']
titles = ['Test Loss', 'Test Accuracy', 'Test AUC']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    values = [optimizer_results[opt][metric] for opt in optimizers_to_test.keys()]
    axes[idx].bar(optimizers_to_test.keys(), values, edgecolor='black', alpha=0.7)
    axes[idx].set_ylabel(metric.upper(), fontsize=12)
    axes[idx].set_title(f'{title} by Optimizer', fontsize=14, fontweight='bold')
    axes[idx].grid(axis='y', alpha=0.3)
    
    for i, v in enumerate(values):
        axes[idx].text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 10. Feature Importance Analysis

In [ ]:
# Approximate feature importance using permutation
from sklearn.inspection import permutation_importance

# Create a wrapper for keras model to work with sklearn
def model_predict(X):
    return improved_model.predict(X).ravel()

# This is a simplified approach - for full analysis, use SHAP or similar
print("Feature names and their indices:")
for idx, col in enumerate(X.columns):
    print(f"{idx}: {col}")

## 11. Save Results

In [ ]:
# Save predictions
results_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted_Baseline': y_pred_baseline.ravel(),
    'Predicted_Improved': y_pred_improved.ravel(),
    'Probability_Baseline': y_pred_proba_baseline.ravel(),
    'Probability_Improved': y_pred_proba_improved.ravel()
})

results_path = '../../../data/outputs/churn_predictions.csv'
results_df.to_csv(results_path, index=False)
print(f"Predictions saved to: {results_path}")

# Save model comparison
comparison_df = pd.DataFrame({
    'Model': ['Baseline', 'Improved'] + list(optimizers_to_test.keys()),
    'Loss': [baseline_results[0], improved_results[0]] + 
            [optimizer_results[opt]['loss'] for opt in optimizers_to_test.keys()],
    'Accuracy': [baseline_results[1], improved_results[1]] + 
                [optimizer_results[opt]['accuracy'] for opt in optimizers_to_test.keys()],
    'AUC': [baseline_results[2], improved_results[2]] + 
           [optimizer_results[opt]['auc'] for opt in optimizers_to_test.keys()]
})

comparison_path = '../../../data/outputs/model_comparison.csv'
comparison_df.to_csv(comparison_path, index=False)
print(f"Model comparison saved to: {comparison_path}")

print("\nAll results saved successfully!")

## 12. Summary and Key Findings

In [ ]:
print("="*80)
print("SESSION 9 SUMMARY: NEURAL NETWORKS FOR CHURN PREDICTION")
print("="*80)

print("\n1. DATASET")
print(f"   - Total customers: {len(df):,}")
print(f"   - Features: {X.shape[1]}")
print(f"   - Churn rate: {df['Exited'].mean()*100:.2f}%")
print(f"   - Train/Test split: {len(X_train)}/{len(X_test)}")

print("\n2. MODELS DEVELOPED")
print("   a) Baseline Model:")
print("      - 3 dense layers (64, 32, 16 neurons)")
print("      - ReLU activation")
print(f"      - Test Accuracy: {baseline_results[1]:.4f}")
print(f"      - Test AUC: {baseline_results[2]:.4f}")

print("\n   b) Improved Model:")
print("      - 4 dense layers (128, 64, 32, 16 neurons)")
print("      - Batch Normalization after each layer")
print("      - Dropout (0.3, 0.3, 0.2, 0.2)")
print(f"      - Test Accuracy: {improved_results[1]:.4f}")
print(f"      - Test AUC: {improved_results[2]:.4f}")

print("\n3. TRAINING TECHNIQUES")
print("   - EarlyStopping: Prevent overfitting")
print("   - ModelCheckpoint: Save best model")
print("   - ReduceLROnPlateau: Adaptive learning rate")
print("   - Batch Normalization: Stabilize training")
print("   - Dropout: Regularization")

print("\n4. OPTIMIZER COMPARISON")
for opt_name in optimizers_to_test.keys():
    print(f"   - {opt_name}: Acc={optimizer_results[opt_name]['accuracy']:.4f}, "
          f"AUC={optimizer_results[opt_name]['auc']:.4f}")

best_optimizer = max(optimizer_results.keys(), 
                    key=lambda x: optimizer_results[x]['auc'])
print(f"\n   Best Optimizer: {best_optimizer}")

print("\n5. KEY INSIGHTS")
print("   - Dropout and BatchNorm significantly improve generalization")
print("   - Model achieves strong AUC score for churn prediction")
print("   - Age, balance, and number of products are key predictors")
print("   - Geography shows significant impact on churn")

print("\n6. BUSINESS IMPACT")
print("   - Early identification of at-risk customers")
print("   - Targeted retention campaigns")
print("   - Estimated cost savings from reduced churn")
print("   - Personalized customer engagement strategies")

print("\n7. FILES GENERATED")
print("   - best_churn_model.keras: Best performing model")
print("   - churn_predictions.csv: Test set predictions")
print("   - model_comparison.csv: Performance metrics")

print("\n" + "="*80)
print("Neural network training complete! Model ready for deployment.")
print("="*80)